# [MOTrainer: Measurement Operator Trainer](https://zenodo.org/record/7540443)

An example notebook that split data to train-test and and uses in DNN trainings.

## Import libraries and set paths

In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt # only for plots
import pickle
import pandas as pd
import logging
import sys
import motrainer
import dask_ml.model_selection as dcv
from motrainer.jackknife import JackknifeGPI

2023-11-24 15:03:15.086175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


data saved from the notebook [1_example_one_pickle_file_with_nested_fields](./1_example_one_pickle_file_with_nested_fields.ipynb)

In [2]:
# Read the data
zarr_file_path = "./example1_data.zarr"
ds = xr.open_zarr(zarr_file_path)
ds

<xarray.Dataset>
Dimensions:    (space: 5, time: 8506)
Coordinates:
    latitude   (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    longitude  (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
Dimensions without coordinates: space
Data variables:
    BIOMA1     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    BIOMA2     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    curv       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    sig        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    slop       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
Attributes:
    license:  data license
    source:   data source

In [3]:
bags = motrainer.dataset_split(ds, "space")
bags

dask.bag<from_sequence, npartitions=5>

In [4]:
def to_dataframe(ds):
    return ds.to_dask_dataframe()

def chunk(ds, chunks):
    return ds.chunk(chunks)

bags = bags.map(chunk, {"space": 100}).map(to_dataframe)
bags

dask.bag<to_dataframe, npartitions=5>

In [5]:
test_size = 0.33
f_shuffle = True
train_test_bags = bags.map(
    dcv.train_test_split, test_size=test_size, shuffle=f_shuffle, random_state=1
)  
train_bags = train_test_bags.pluck(0)
test_bags = train_test_bags.pluck(1)

In [6]:
# path to store results
out_path = './dnn_examples/results'

# JackKnife parameters
val_split_year = 2017
output_list = ['sig', 'slop', 'curv']
input_list = ['TG1', 'TG2', 'TG3', 'WG1', 'WG2', 'WG3', 'BIOMA1', 'BIOMA2']

# Training parameters
searching_space = {
    'num_dense_layers': [1, 10],
    'num_input_nodes': [1, 6],
    'num_dense_nodes': [1, 128],
    'learning_rate': [5e-4, 1e-2],
    'activation': ['relu']
}
optimize_space = {
    'best_loss': 2, # 1
    'n_calls': 11, # 15
    'epochs': 5, # 300
    'noise': 0.1, # 0.1
    'n_jobs': 4,  # -1 NO. of cores
    'kappa': 5,
    'validation_split': 0.2,
    'x0': [1e-3, 1, 4, 13, 'relu', 64]
} # For weightling loss: 'loss_weights': [1, 1, 0.5], 

In [7]:
# disable WARNING:absl:Found untraced functions such as _update_step_xla while saving
# see https://github.com/tensorflow/tensorflow/issues/47554
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [11]:
aprior, post = [], []
# Loop all gpi
for gpi_num, df in enumerate(train_bags):

    # remove NA data
    gpi_data = df.compute()
    gpi_data.dropna(inplace=True)

    # add year to index
    gpi_data["year"] = gpi_data["time"].dt.year
    gpi_data.set_index("year", append=True, inplace=True, drop=True)

    gpi = JackknifeGPI(gpi_data,
                       val_split_year,
                       input_list,
                       output_list,
                       outpath=f'{out_path}/gpi{gpi_num+1}')

    gpi.train(searching_space=searching_space,
              optimize_space=optimize_space,
              normalize_method='standard',
              training_method='dnn',
              performance_method='rmse',
              verbose=2)

    gpi.export_best()

    # Export apriori performance
    path_apriori_performance = f'{gpi.outpath}/apriori_performance_{gpi.best_year}'
    with open(path_apriori_performance, 'wb') as f:
        pickle.dump(gpi.apr_perf, f)

    # Export postpriori performance
    path_postpriori_performance = f'{gpi.outpath}/postpriori_performance_{gpi.best_year}'
    with open(path_postpriori_performance, 'wb') as f:
        pickle.dump(gpi.post_perf, f)

    aprior.append(gpi.apr_perf)
    post.append(gpi.post_perf)
    print("=========================================")
    print(f"       GPI {gpi_num+1} done")
    print("       aprior performance(RMSE): ")
    print(gpi.apr_perf)
    print("=========================================")
    print("       post performance(RMSE): ")
    print(gpi.post_perf)
    print("=========================================")
print("Training is finished!")

Epoch 1/5
39/39 - 1s - loss: 0.9694 - mae: 0.8016 - acc: 0.2985 - val_loss: 0.9144 - val_mae: 0.7827 - val_acc: 0.3421 - 565ms/epoch - 14ms/step
Epoch 2/5
39/39 - 0s - loss: 0.8363 - mae: 0.7416 - acc: 0.3763 - val_loss: 0.8230 - val_mae: 0.7373 - val_acc: 0.4714 - 54ms/epoch - 1ms/step
Epoch 3/5
39/39 - 0s - loss: 0.7444 - mae: 0.6929 - acc: 0.5397 - val_loss: 0.7378 - val_mae: 0.6899 - val_acc: 0.6399 - 54ms/epoch - 1ms/step
Epoch 4/5
39/39 - 0s - loss: 0.6788 - mae: 0.6526 - acc: 0.6495 - val_loss: 0.6879 - val_mae: 0.6603 - val_acc: 0.6579 - 56ms/epoch - 1ms/step
Epoch 5/5
39/39 - 0s - loss: 0.6456 - mae: 0.6316 - acc: 0.6499 - val_loss: 0.6658 - val_mae: 0.6457 - val_acc: 0.6612 - 56ms/epoch - 1ms/step
Epoch 1/5
10/10 - 1s - loss: 0.9957 - mae: 0.8026 - acc: 0.3993 - val_loss: 0.9436 - val_mae: 0.7849 - val_acc: 0.4321 - 542ms/epoch - 54ms/step
Epoch 2/5
10/10 - 0s - loss: 0.8694 - mae: 0.7443 - acc: 0.4697 - val_loss: 0.8598 - val_mae: 0.7413 - val_acc: 0.5106 - 32ms/epoch - 3ms/

INFO:tensorflow:Assets written to: ./dnn_examples/results/gpi1/all_years/optimized_model_2014/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model4948580026381091920/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model4948580026381091920/assets


16/16 [==============================] - 0s 771us/step
INFO:tensorflow:Assets written to: /tmp/tmp_model13960991189797199493/assets


...

17/17 [==============================] - 0s 886us/step
Epoch 1/5
45/45 - 1s - loss: 1.0850 - mae: 0.8454 - acc: 0.4508 - val_loss: 0.9736 - val_mae: 0.8019 - val_acc: 0.6166 - 580ms/epoch - 13ms/step
Epoch 2/5
45/45 - 0s - loss: 0.9165 - mae: 0.7657 - acc: 0.6211 - val_loss: 0.8481 - val_mae: 0.7399 - val_acc: 0.6657 - 63ms/epoch - 1ms/step
Epoch 3/5
45/45 - 0s - loss: 0.8090 - mae: 0.7044 - acc: 0.6573 - val_loss: 0.7556 - val_mae: 0.6820 - val_acc: 0.6840 - 65ms/epoch - 1ms/step
Epoch 4/5
45/45 - 0s - loss: 0.7318 - mae: 0.6592 - acc: 0.6752 - val_loss: 0.6914 - val_mae: 0.6466 - val_acc: 0.6938 - 62ms/epoch - 1ms/step
Epoch 5/5
45/45 - 0s - loss: 0.6767 - mae: 0.6333 - acc: 0.6770 - val_loss: 0.6538 - val_mae: 0.6275 - val_acc: 0.6980 - 65ms/epoch - 1ms/step
Epoch 1/5
14/14 - 1s - loss: 0.9850 - mae: 0.8027 - acc: 0.4298 - val_loss: 0.9384 - val_mae: 0.7872 - val_acc: 0.6110 - 1s/epoch - 80ms/step
Epoch 2/5
14/14 - 0s - loss: 0.8234 - mae: 0.7185 - acc: 0.6134 - val_loss: 0.7158 - v

INFO:tensorflow:Assets written to: ./dnn_examples/results/gpi1/all_years/optimized_model_2017/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model11812396721837904069/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model11812396721837904069/assets


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.